<a href="https://colab.research.google.com/github/sysadminamit/HelpfulnessPrediction-of-Online-Review/blob/master/TFIDFSVD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this Notebook I have created TFIDF score based on my dataset and create a csv file from that for further use of this as document vector

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
from google.colab import files
uploaded = files.upload()

Saving AmazonElectronicsProductReviewClean.csv to AmazonElectronicsProductReviewClean (1).csv


In [0]:
import pandas as pd
df=pd.read_csv('gdrive/My Drive/AmazonElectronicsProductReviewClean.csv')

In [0]:
df.head(5)

,Unnamed: 0,overall,Helpfulfactor,review,Summary
0,0,1,1.0,excite especially since every time refuse turn...,very disappointed
1,1,3,1.0,enter multiple stop vias nowhere near userfrie...,1st impression
2,2,2,1.0,might think conjested city situation might get...,great grafics poor gps
3,3,5,1.0,hd picture samsung tv excellent,hdmi nook adapter cable
4,4,5,1.0,warn,this works great but read the details


In [0]:
import matplotlib.pyplot as plt
import seaborn as sns


In [0]:
# Starting with the CountVectorizer/TfidfTransformer approach...
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

cvec = CountVectorizer(stop_words='english', min_df=1, max_df=.5, ngram_range=(1,2))
cvec

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=0.5, max_features=None, min_df=1,
                ngram_range=(1, 2), preprocessor=None, stop_words='english',
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [0]:
# Calculate all the n-grams found in all documents
from itertools import islice
cvec.fit_transform(df['review'].values.astype('U'))
list(islice(cvec.vocabulary_.items(), 20))

[('excite', 303936),
 ('especially', 296841),
 ('time', 962755),
 ('refuse', 774069),
 ('turn', 988674),
 ('street', 912441),
 ('post', 702106),
 ('truck', 983496),
 ('tnd', 967922),
 ('minute', 573275),
 ('figure', 330743),
 ('reroute', 786898),
 ('happen', 395881),
 ('multiple', 593534),
 ('short', 846949),
 ('trip', 981133),
 ('excite especially', 303966),
 ('especially time', 297562),
 ('time refuse', 964468),
 ('refuse turn', 774148)]

In [0]:
# Check how many total n-grams we have
len(cvec.vocabulary_)

1097218

In [0]:
# Initialize the vectorizer with new settings and check the new vocabulary length
cvec = CountVectorizer(stop_words='english', min_df=.0025, max_df=.1, ngram_range=(1,2))
cvec.fit_transform(df['review'].values.astype('U'))
len(cvec.vocabulary_)

515

In [0]:
cvec_counts = cvec.fit_transform(df['review'].values.astype('U'))
print('sparse matrix shape:', cvec_counts.shape)
print ('nonzero count:', cvec_counts.nnz)
print ('sparsity: %.2f%%' % (100.0 * cvec_counts.nnz / (cvec_counts.shape[0] * cvec_counts.shape[1])))

sparse matrix shape: (443252, 515)
nonzero count: 1828533
sparsity: 0.80%


In [0]:
transformer = TfidfTransformer()
transformed_weights = transformer.fit_transform(cvec_counts)
#transformed_weight = transformer.fit_transform(df['review'])
transformed_weights

<443252x515 sparse matrix of type '<class 'numpy.float64'>'
	with 1828533 stored elements in Compressed Sparse Row format>

In [0]:
import numpy as np
weights = np.asarray(transformed_weights.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'term': cvec.get_feature_names(), 'weight': weights})
weights_df.sort_values(by='weight', ascending=False).head(20)

,term,weight
168,great,0.030941
164,good,0.026731
48,buy,0.018496
281,need,0.017581
339,product,0.017429
49,cable,0.016836
335,price,0.016765
239,like,0.015738
258,make,0.015153
51,camera,0.014732


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
tvec = TfidfVectorizer(min_df=.0025, max_df=.1, stop_words='english', ngram_range=(1,2))
tvec_weights = tvec.fit_transform(df.review.dropna())
weights = np.asarray(tvec_weights.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'term': tvec.get_feature_names(), 'weight': weights})
weights_df.sort_values(by='weight', ascending=False).head(20)

,term,weight
168,great,0.030986
164,good,0.026769
48,buy,0.018526
282,need,0.017604
340,product,0.017457
49,cable,0.016857
336,price,0.016791
239,like,0.015756
259,make,0.015173
51,camera,0.014746


In [0]:
tvec.get_feature_names()

In [0]:
tfidfdata = pd.DataFrame(tvec_weights.toarray(), columns=tvec.get_feature_names())
tfidfdata

,able,access,accessory,actually,adapter,add,adjust,advertised,ago,air,allow,amaze,amazing,amazon,amp,angle,antenna,appear,apple,area,arrive,ask,attach,audio,available,away,awesome,backup,bad,bag,base,basic,bass,battery,beat,best,big,bit,black,blue,...,view,volume,wait,wall,want,warranty,wasnt,waste,watch,way,wear,week,weight,white,wide,wife,wifi,window,wire,wireless,wish,wonderful,wont,work advertised,work fine,work good,work great,work like,work perfectly,worry,worth,wouldnt,wow,write,wrong,year,yes,youll,youre,zoom
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0000,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0000,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0000,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0000,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0000,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0000,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0000,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.487832,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0000,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.

In [0]:
from scipy.linalg import svd as scipy_svd
from sklearn.pipeline import Pipeline
from sklearn.datasets import load_iris
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.preprocessing import normalize, StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer

In [0]:
svdT = TruncatedSVD(n_components=100)
svdTFit = svdT.fit_transform(tfidfdata)

In [0]:
svdTFit

array([[ 0.04729389, -0.05239056, -0.05865905, ..., -0.0027654 ,
         0.0276494 ,  0.06550677],
       [ 0.01938248, -0.01862148, -0.02098117, ...,  0.05003855,
        -0.04425171,  0.08529483],
       [ 0.06120604, -0.056924  , -0.0420643 , ..., -0.02172337,
         0.02112038,  0.0157726 ],
       ...,
       [ 0.1109949 , -0.18655585,  0.18012424, ..., -0.02933536,
        -0.02103804,  0.05595103],
       [ 0.03524694, -0.041377  , -0.01745855, ...,  0.10702993,
         0.06971408,  0.01261282],
       [ 0.09413147, -0.10105388,  0.03636358, ..., -0.04343917,
        -0.00410995, -0.06627083]])

In [0]:
svddata = pd.DataFrame(svdTFit)
svddata

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
0,4.729389e-02,-5.239056e-02,-5.865905e-02,-5.510956e-02,-1.676346e-03,-3.967563e-02,-8.260042e-03,-9.231128e-03,-2.905627e-02,3.931114e-02,3.420360e-02,-4.457667e-02,1.913136e-02,1.704947e-01,-1.828297e-01,-7.886883e-03,4.768163e-02,1.263137e-01,-2.257553e-01,8.380252e-02,-1.303770e-01,-9.418585e-02,5.160054e-02,-9.847182e-02,4.654787e-03,2.457781e-02,-8.237938e-03,-1.724450e-02,3.905894e-03,-2.316763e-02,5.955801e-03,4.911119e-04,-2.285798e-02,8.304447e-04,2.160660e-02,-2.721182e-02,-8.138747e-03,1.741634e-03,2.405134e-02,1.848177e-02,...,-2.124430e-03,3.513649e-03,1.519499e-02,2.666663e-02,-5.142583e-02,-4.447503e-02,5.806075e-03,1.353770e-03,2.055299e-02,-1.119799e-03,-2.478427e-02,-1.361116e-02,-3.130936e-02,1.001339e-02,1.480914e-02,1.103600e-03,-1.716682e-03,-2.510960e-02,-5.957714e-03,-1.149295e-02,-4.007210e-03,-4.106179e-02,-1.176046e-02,4.796023e-03,2.172463e-02,1.517455e-02,-3.256539e-02,4.193458e-02,7.161039e-02,3.113491e-02,4.184247e-02,4.975988e-02,-3.571384e-02,-1.300244e-02,8.308873e-03,-5.662774e-02,-1.357881e-02,-2.765399e-03,2.764940e-02,6.550677e-02
1,1.938248e-02,-1.862148e-02,-2.098117e-02,-1.544186e-02,-3.813804e-03,-1.073766e-02,-9.528898e-04,-7.734874e-03,-5.685014e-03,8.077107e-03,7.374899e-03,-1.139946e-02,3.464214e-03,2.078735e-02,-3.718583e-03,1.769905e-03,6.365931e-03,5.666443e-03,-1.179250e-03,-1.654515e-02,1.146922e-02,-1.199565e-03,-4.055942e-03,2.864219e-02,-9.498425e-03,6.023711e-03,-1.360892e-02,6.428952e-03,-1.283217e-03,1.421971e-03,-1.259674e-02,-1.020971e-02,-6.735248e-03,-1.661542e-02,-2.025872e-02,9.386643e-03,2.064950e-02,1.808262e-02,1.583174e-02,1.781745e-03,...,-2.607298e-02,-3.983932e-03,1.627703e-02,8.995388e-02,-1.442649e-03,5.192852e-02,-3.826322e-02,-4.055345e-02,-1.111545e-02,1.191847e-02,-5.309504e-02,5.996123e-03,1.084808e-02,-1.739788e-02,-1.605123e-02,-2.501340e-02,4.641563e-02,7.604983e-03,-9.807358e-03,1.522238e-03,1.881681e-02,7.241060e-03,1.306380e-02,-4.459666e-02,-1.246849e-02,-1.800954e-02,-4.451726e-02,1.349150e-01,-9.722329e-02,1.003969e-01,7.576183e-02,-1.176670e-01,5.707799e-02,3.034724e-02,1.419790e-01,1.676340e-02,3.569250e-02,5.003855e-02,-4.425171e-02,8.529483e-02
2,6.120604e-02,-5.692400e-02,-4.206430e-02,-5.670435e-02,-1.108194e-02,-4.617407e-02,-2.978725e-02,5.671865e-03,1.025487e-02,2.731672e-04,1.408658e-03,-2.410289e-02,2.199532e-02,1.793021e-02,1.643067e-02,-7.535824e-03,6.505938e-02,4.834155e-02,-2.819912e-02,2.013237e-02,3.707367e-02,-2.719207e-02,-1.032248e-01,1.025874e-01,-4.812386e-02,-8.141229e-02,1.883157e-01,-8.295929e-02,1.069603e-01,4.598433e-01,2.618263e-01,1.527664e-01,-7.977829e-02,8.879715e-02,-2.289147e-02,-9.790289e-04,1.228040e-02,-1.791821e-02,3.372925e-03,-4.910802e-02,...,-2.463067e-02,-5.322833e-03,4.114785e-02,4.040600e-02,1.341518e-02,-3.229149e-02,3.237734e-02,3.314632e-02,-2.236841e-03,-8.635213e-02,-1.584153e-01,2.089488e-01,-9.302616e-02,1.666180e-01,-3.128975e-01,2.850253e-01,1.425037e-01,-1.258338e-01,1.005745e-01,-1.075946e-01,-6.895563e-03,2.394169e-01,1.065566e-02,-7.102484e-02,8.958401e-02,-1.192600e-01,-7.741804e-02,-7.432929e-04,-3.784568e-02,-7.248519e-02,4.847492e-02,-3.563801e-02,-4.339392e-02,5.289278e-02,-3.396523e-02,-1.598509e-02,-4.449265e-02,-2.172337e-02,2.112038e-02,1.577260e-02
3,5.619478e-02,-4.827798e-02,-2.113477e-02,-3.880640e-02,-9.915602e-03,1.125213e-02,-3.356831e-02,-1.792852e-02,2.201732e-02,-4.422398e-02,2.905331e-02,3.213551e-02,-1.271316e-02,1.715887e-02,2.929616e-03,-2.434425e-03,-2.199355e-02,4.581165e-02,-1.097153e-02,-8.438506e-03,5.123109e-03,3.367645e-02,4.521143e-02,7.354779e-02,-2.716333e-02,5.433501e-02,2.693693e-02,7.889717e-02,3.463791e-02,3.388596e-02,-9.327442e-02,-7.507806e-02,-6.002824e-02,-5.139969e-02,-1.058840e-01,3.859846e-02,7.377170e-02,-9.